In [1]:
import numpy as np
np.bool = np.bool_

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from gluonts.dataset.common import ListDataset
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
import mxnet as mx

In [2]:
## Read the CSV file
#file_path_1 = 'C:\Theo\School\ESIEE\Annee\E5\TimeSeries\Projet DeepAR\Dataset\ETT-small\ETTh1.csv'
#file_path_2 = 'C:\Theo\School\ESIEE\Annee\E5\TimeSeries\Projet DeepAR\Dataset\ETT-small\ETTh2.csv'
#file_path_3 = 'C:\Theo\School\ESIEE\Annee\E5\TimeSeries\Projet DeepAR\Dataset\ETT-small\ETTm1.csv'
#file_path_4 = 'C:\Theo\School\ESIEE\Annee\E5\TimeSeries\Projet DeepAR\Dataset\ETT-small\ETTm2.csv'
#
#df_1 = pd.read_csv(file_path_1)
#df_2 = pd.read_csv(file_path_2)
#df_3 = pd.read_csv(file_path_3)
#df_4 = pd.read_csv(file_path_4)

In [3]:
#print(df_1.shape)
#print(df_2.shape)
#print(df_3.shape)
#print(df_4.shape)

In [4]:
#display(df_4)

In [5]:
file_path_weather = 'D:\Ce PC\Documents\Times seriezs\Time_series-1\Dataset\weather\weather.csv'

df_weather = pd.read_csv(file_path_weather)

In [ ]:
display(df_weather)

In [7]:
## Convert 'date' column to datetime
#df_electricity['date'] = pd.to_datetime(df_electricity['date'])
#
## Select all columns except 'date' to plot
#columns_to_plot = df_electricity.columns.drop('date')
#
## Plot all columns against the date
#df_electricity.plot(x='date', y=columns_to_plot, figsize=(10,6), legend=False)
#
## Customize the plot
#plt.title('All Data over Time')
#plt.xlabel('Date')
#plt.ylabel('Values')
#plt.grid(True)
#plt.tight_layout()
#
## Show the plot
#plt.show()

In [8]:
# Conversion de la colonne 'date' en format datetime
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%Y-%m-%d %H:%M:%S')

In [9]:
# Création d'une colonne 'season' basée sur le mois
def get_season(month):
    if month in [12, 1, 2]:
        return 0
    elif month in [3, 4, 5]:
        return 1
    elif month in [6, 7, 8]:
        return 2
    else:
        return 3

In [10]:
# Création d'une colonne 'wind_chill'
def calculate_wind_chill(temp, wind_speed):
    if temp <= 10 and wind_speed > 4.8:
        return 13.12 + 0.6215 * temp - 11.37 * (wind_speed ** 0.16) + 0.3965 * temp * (wind_speed ** 0.16)
    else:
        return temp

In [11]:
# Création de caractéristiques temporelles à partir de la colonne 'date'
df_weather['month'] = df_weather['date'].dt.month
df_weather['day_of_week'] = df_weather['date'].dt.dayofweek
df_weather['hour'] = df_weather['date'].dt.hour
df_weather['season'] = df_weather['month'].apply(get_season)
df_weather['week_of_year'] = df_weather['date'].dt.isocalendar().week

In [12]:
# Amplitude journalière de la température (différence entre max et min température sur une journée)
df_weather['temp_daily_amplitude'] = df_weather['T (degC)'].rolling(window=24).max() - df_weather['T (degC)'].rolling(window=24).min()

# Différence de température d'une heure à l'autre
df_weather['temp_delta'] = df_weather['T (degC)'].diff(periods=1)

# Lag Features (température des dernières 1, 3, et 6 heures)
df_weather['temp_lag_1h'] = df_weather['T (degC)'].shift(1)
df_weather['temp_lag_3h'] = df_weather['T (degC)'].shift(3)
df_weather['temp_lag_6h'] = df_weather['T (degC)'].shift(6)

# Interaction entre VPmax et Tdew
df_weather['VPmax_Tdew_interaction'] = df_weather['VPmax (mbar)'] * df_weather['Tdew (degC)']

# Indice thermique (approximation simple avec température et humidité)
df_weather['heat_index'] = 0.5 * (df_weather['T (degC)'] + 61.0 + ((df_weather['T (degC)'] - 68.0) * 1.2) + (df_weather['rh (%)'] * 0.094))

# Indice de confort (Wind Chill)
df_weather['wind_chill'] = df_weather.apply(lambda row: calculate_wind_chill(row['T (degC)'], row['wv (m/s)'] * 3.6), axis=1)

In [13]:
# Définir une fenêtre de 3 heures (ou 3 périodes dans notre cas)
window_size = 3

# Créer des features rolling mean et rolling std pour les colonnes pertinentes
df_weather['Tdew_rolling_mean'] = df_weather['Tdew (degC)'].rolling(window=window_size).mean()
df_weather['Tdew_rolling_std'] = df_weather['Tdew (degC)'].rolling(window=window_size).std()

df_weather['VPmax_rolling_mean'] = df_weather['VPmax (mbar)'].rolling(window=window_size).mean()
df_weather['VPmax_rolling_std'] = df_weather['VPmax (mbar)'].rolling(window=window_size).std()

df_weather['rho_rolling_mean'] = df_weather['rho (g/m**3)'].rolling(window=window_size).mean()
df_weather['rho_rolling_std'] = df_weather['rho (g/m**3)'].rolling(window=window_size).std()


In [14]:
# Supprimer les lignes avec des NaN dans les colonnes avec des valeurs manquantes
df_weather = df_weather.dropna()

In [ ]:
# Vérifier les valeurs manquantes dans le dataset
missing_values = df_weather.isnull().sum()

# Afficher les résultats
missing_values

In [ ]:
# Afficher les premières lignes pour vérifier les nouvelles colonnes
df_weather.columns

In [ ]:
# Graphique 1: Température en fonction de la colonne date
plt.figure(figsize=(10, 5))
plt.plot(df_weather['date'], df_weather['T (degC)'], label='Température (°C)', color='b')
plt.xlabel('Date')
plt.ylabel('Température (°C)')
plt.title('Température en fonction de la date')
plt.grid(True)
plt.show()


In [ ]:
# Graphique 2: Température en fonction de la colonne hour
plt.figure(figsize=(10, 5))
hourly_temp = df_weather.groupby('hour')['T (degC)'].mean()
plt.plot(hourly_temp.index, hourly_temp.values, marker='o', linestyle='-', color='g')
plt.xlabel('Heure')
plt.ylabel('Température Moyenne (°C)')
plt.title('Température en fonction de l\'heure de la journée')
plt.grid(True)
plt.show()

In [ ]:
# Graphique 3: Température en fonction du jour de la semaine
plt.figure(figsize=(10, 5))
weekly_temp = df_weather.groupby('day_of_week')['T (degC)'].mean()
plt.bar(weekly_temp.index, weekly_temp.values, color='c', edgecolor='black')
plt.xlabel('Jour de la semaine')
plt.ylabel('Température Moyenne (°C)')
plt.title('Température en fonction du jour de la semaine')
plt.xticks(range(0, 7), ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'])
plt.grid(axis='y')
plt.show()

In [ ]:
# Graphique 4: Température en fonction du mois
plt.figure(figsize=(10, 5))
monthly_temp = df_weather.groupby('month')['T (degC)'].mean()
plt.plot(monthly_temp.index, monthly_temp.values, marker='o', linestyle='-', color='r')
plt.xlabel('Mois')
plt.ylabel('Température Moyenne (°C)')
plt.title('Température en fonction du mois')
plt.grid(True)
plt.xticks(range(1, 13))
plt.show()

In [ ]:
# Graphique 5: Température en fonction de la saison
plt.figure(figsize=(10, 5))
seasonal_temp = df_weather.groupby('season')['T (degC)'].mean()
plt.bar(seasonal_temp.index, seasonal_temp.values, color='purple', edgecolor='black')
plt.xlabel('Saison')
plt.ylabel('Température Moyenne (°C)')
plt.title('Température en fonction de la saison')
plt.grid(axis='y')
plt.show()

In [ ]:
# Graphique 6: Température en fonction de la semaine de l'année
plt.figure(figsize=(10, 5))
weekly_year_temp = df_weather.groupby('week_of_year')['T (degC)'].mean()
plt.plot(weekly_year_temp.index, weekly_year_temp.values, marker='o', linestyle='-', color='orange')
plt.xlabel('Semaine de l\'année')
plt.ylabel('Température Moyenne (°C)')
plt.title('Température en fonction de la semaine de l\'année')
plt.grid(True)
plt.show()

In [ ]:
# Calcul de la matrice de corrélation avec toutes les colonnes disponibles
correlation_matrix_full = df_weather.corr()

# Afficher la matrice de corrélation complète
plt.figure(figsize=(12, 10))
plt.matshow(correlation_matrix_full, cmap='coolwarm', fignum=1)
plt.colorbar()
plt.xticks(range(correlation_matrix_full.shape[1]), correlation_matrix_full.columns, rotation=90)
plt.yticks(range(correlation_matrix_full.shape[1]), correlation_matrix_full.columns)
plt.title('Matrice de corrélation complète', pad=20)
plt.show()

# Afficher la matrice de corrélation complète en version tableau
print(correlation_matrix_full)


# Caractéristiques avec une forte corrélation avec la température :

Tpot (K) : Très forte corrélation (0.995), mais potentiellement redondante avec la température elle-même. Cela pourrait être exclu pour éviter la redondance.

Tdew (degC) : Corrélation élevée (0.783), ce qui indique que la température de rosée a une influence importante sur la température.

VPmax (mbar) : Corrélation forte (0.968), la pression de vapeur maximale est pertinente.

Tlog (degC) : Très forte corrélation (0.982), mais potentiellement redondante avec la température. À évaluer si nécessaire.

VPact (mbar) : Moyenne à forte corrélation (~0.76).

rho (g/m³) : Corrélation négative forte (~-0.95), ce qui en fait une caractéristique intéressante pour la prédiction.

temp_lag_1h : Très forte corrélation (0.999), donc la température avec un décalage d'une heure est très pertinente.

temp_lag_3h et temp_lag_6h : Corrélations élevées (0.996 et 0.989), donc les décalages à 3 et 6 heures sont également utiles.

VPmax_Tdew_interaction : Corrélation forte (0.856), l'interaction entre la pression de vapeur maximale et la température de rosée est pertinente.

heat_index : Très forte corrélation (0.995), indiquant que l'indice thermique est également une bonne mesure.

Tdew_rolling_mean : Corrélation modérée (0.782), la moyenne mobile de la température de rosée est également utile.

VPmax_rolling_mean : Forte corrélation positive (0.968), la moyenne mobile de la pression de vapeur maximale est un bon prédicteur.

rho_rolling_mean : Forte corrélation négative (-0.946), la moyenne mobile de la densité de l'air influence la température de manière inverse.

# Variables temporelles :

month : Corrélation modérée, pertinente pour capturer les variations saisonnières.

hour : Corrélation modérée avec la température, utile pour capturer les variations diurnes.

season et week_of_year : Peuvent être incluses pour aider à capturer les tendances saisonnières.

# Sélection finale pour le modèle DeepAR :

Tdew (degC)

VPmax (mbar)

rho (g/m³)

temp_lag_1h

temp_lag_3h

temp_lag_6h

VPmax_Tdew_interaction

heat_index

Tdew_rolling_mean

VPmax_rolling_mean

rho_rolling_mean

month

hour

week_of_year


In [24]:
# Inclure toutes les features sélectionnées dans la nouvelle dataframe df_final
all_selected_features = ['date', 'month', 'hour', 'week_of_year','Tdew (degC)',
                         'VPmax (mbar)', 'rho (g/m**3)', 'temp_lag_1h',
                         'temp_lag_3h', 'temp_lag_6h', 'VPmax_Tdew_interaction', 
                         'heat_index', 'Tdew_rolling_mean', 'VPmax_rolling_mean',
                         'rho_rolling_mean']

# Créer la nouvelle dataframe df_final avec toutes les caractéristiques sélectionnées
df_final = df_weather[all_selected_features].copy()

# Appliquer la standardisation uniquement aux colonnes pertinentes
features_to_standardize = ['Tdew (degC)', 'VPmax (mbar)', 'rho (g/m**3)', 
                           'temp_lag_1h', 'temp_lag_3h', 'temp_lag_6h', 
                           'VPmax_Tdew_interaction', 'heat_index', 
                           'Tdew_rolling_mean', 'VPmax_rolling_mean',
                           'rho_rolling_mean']

# Recréer le standardiseur
scaler = StandardScaler()

# Standardiser les colonnes pertinentes
df_final[features_to_standardize] = scaler.fit_transform(df_final[features_to_standardize])

In [ ]:
df_final.head()